In [15]:
#Importing libraries

import json
import csv
import matplotlib
import numpy as np
from os import listdir
from sklearn import linear_model
import random
import copy
import operator
import GPy
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn import svm
from sklearn.neural_network import MLPClassifier
#import alignment #my library 


In [2]:
#Consider that maybe instead of computing the distance between strings would be better to match words?
#Or other kinds of similarity computation

#this is necessary because of the accents an other errors from scrapping around
def levenshtein(s1, s2):
    #https://en.wikipedia.org/wiki/Levenshtein_distance
    if len(s1) < len(s2):
        return lev(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j previous_row and current_row are 1 char longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]


def lev(s1,s2):
    #convert to lower everything to avoid mistakes
    return levenshtein(s1.lower(),s2.lower())

In [3]:
#Load classification
classification = np.genfromtxt('/home/roc/workspace/fifa-db-players/final-classification.csv', delimiter=','
                            , dtype="|S20")
classification = {x[0]:x[1] for x in classification} #transforming it into a dictionary


#Load matches, results, bets
match_infos = np.genfromtxt('/home/roc/workspace/fifa-db-players/match-results.csv', delimiter=',', skip_header=1 
                     ,dtype="|S20")


In [4]:
#Prediction ability based on the betting sites
hits = np.zeros(13)
for row in match_infos:
    h = int(row[4])
    a = int(row[5])
    for i in range(13):
        if h > a : #home wins
            if row[23+i*3+1] >= row[23+i*3] and row[23+i*3] <= row[23+i*3+2]:
                hits[i] = hits[i]+1
        if h < a : #away wins
            if row[23+i*3+2] <= row[23+i*3] and row[23+i*3+1] >= row[23+i*3+2]:
                hits[i] = hits[i]+1
        if h == a : #draw
            if row[23+i*3+1] <= row[23+i*3] and row[23+i*3+1] <= row[23+i*3+2]:
                hits[i] = hits[i]+1
for hit in hits:
    print 'Accuracy: ',hit*100.0/len(match_infos),'%'

Accuracy:  47.6315789474 %
Accuracy:  48.4210526316 %
Accuracy:  48.9473684211 %
Accuracy:  47.8947368421 %
Accuracy:  47.6315789474 %
Accuracy:  48.1578947368 %
Accuracy:  47.8947368421 %
Accuracy:  27.3684210526 %
Accuracy:  32.3684210526 %
Accuracy:  25.7894736842 %
Accuracy:  39.2105263158 %
Accuracy:  28.1578947368 %
Accuracy:  36.5789473684 %


In [5]:
#Prediction ability based on the classification

def get_position(team):
    min_lev = [30, 99] #position, distance lev
    for key,t in classification.iteritems():
        dist = lev(team,t)
        if dist<= min_lev[1]:
            min_lev[0] = key
            min_lev[1] = dist
    return int(min_lev[0])

hits = 0
for match in match_infos:
    th = match[2]
    ta = match[3]
    gh = int(match[4])
    ga = int(match[5])
    
    if gh > ga: #home wins
        if get_position(th) < get_position(ta):
            hits = hits + 1
    if gh < ga: #away wins
        if get_position(th) > get_position(ta):
            hits = hits + 1
    #'''
    if gh == ga:
        if get_position(th)+1 == get_position(ta) or get_position(th) == get_position(ta)+1:
            hits = hits+1
    #'''

print 'Accuracy: ',hits*100.0/len(match_infos),'%'


Accuracy:  54.4736842105 %


In [6]:
#Getting the clubs relevant for this computation
clubs = []
c_names = listdir('/home/roc/workspace/fifa-db-players/club-data')
for key, value in classification.iteritems():
    min_lev = [30, 99] #position, distance lev
    for c in c_names:
        dist = lev(value,c[:-5])
        if dist<= min_lev[1]:
            min_lev[0] = c
            min_lev[1] = dist
    clubs.append(min_lev[0])



In [7]:
#preparing the dataset: teams, results, players (to be replaced by values)

def get_file(team):
    min_lev = [30, 99] #position, distance lev
    for i in range(len(clubs)):
        dist = lev(team,clubs[i])
        if dist<= min_lev[1]:
            min_lev[0] = clubs[i]
            min_lev[1] = dist
    return min_lev[0]



matches = listdir('/home/roc/workspace/fifa-db-players/result-lineup/')

#N_samples x N_features

training = np.zeros((len(matches),133)) #133 = 22 (players) x 6 (6 feartures per player)+1 (the Y)
i = 0
for match in matches:
    teams = str.split(match[:-5],'-v-')
    with open('/home/roc/workspace/fifa-db-players/result-lineup/'+match) as data_file:    
        match_lineup = data_file.read()
    match_lineup = str.split(match_lineup,'\\t')
    match_lineup[0] = match_lineup[0][1:]
    match_lineup[len(match_lineup)-1] = match_lineup[len(match_lineup)-1][:-1]
    players = match_lineup[5:]

    with open('/home/roc/workspace/fifa-db-players/club-data/'+get_file(teams[0])) as data_file:    
        team_info = json.load(data_file)

    for player in range(len(players)):
        if player>=11: #check that the change has been done where it is required
            with open('/home/roc/workspace/fifa-db-players/club-data/'+get_file(teams[1])) as data_file:    
                team_info = json.load(data_file)
        
        #find the closest player in team_info (not perfect, it might be not inside)
        min_lev = [0, 999] #position, distance lev
        for key, value in team_info.iteritems():
            dist = lev(players[player], team_info[key]['name'])#lev(team,clubs[i])
            if dist<= min_lev[1]:
                min_lev[0] = key
                min_lev[1] = dist
                
        #load ability in the training matrix
        training[i][player*6+0] = team_info[min_lev[0]]["abilities"]['pas']
        training[i][player*6+1] = team_info[min_lev[0]]["abilities"]['dri']
        training[i][player*6+2] = team_info[min_lev[0]]["abilities"]['phy']
        training[i][player*6+3] = team_info[min_lev[0]]["abilities"]['pac']
        training[i][player*6+4] = team_info[min_lev[0]]["abilities"]['deff']
        training[i][player*6+5] = team_info[min_lev[0]]["abilities"]['sho']

    training[i][22*6] = int(match_lineup[4])
        
    i = i+1
print 'training'

training


In [8]:
#Separating the train and test datasets
random.seed(8)

index = random.sample(xrange(len(matches)), int(len(matches)*0.2)) #20% of testing

test = training[index,:]
train = np.delete(training, index,0)

#More dataset separation
X_train = train[:,:len(train[0])-1]
Y_train = train[:,len(train[0])-1]

X_test = test[:,:len(train[0])-1]
Y_test = test[:,len(train[0])-1]

In [ ]:
#computing accuracy
def compute_accuracy(prediction, Y_test):
    threshold = 0.3
    accuracy = 0
    for i in range(len(prediction)):
        if prediction[i]-threshold > 0 and Y_test[i]>0: 
            accuracy = accuracy+1
        elif prediction[i]+threshold < 0 and Y_test[i]<0: 
            accuracy = accuracy+1
        elif Y_test[i]+threshold>prediction[i] and Y_test[i]-1*threshold<prediction[i]: 
            accuracy = accuracy+1
    return accuracy

In [21]:
#Do the linear regression

regr = linear_model.LinearRegression()
regr.fit(X_train,Y_train)
prediction = regr.predict(X_test)
prediction_linear = regr.predict(X_train)
print 'accuracy', compute_accuracy(prediction, Y_test)*100/len(prediction),'%'


accuracy 46 %


In [22]:
#Do gaussian process
kernel = GPy.kern.RBF(1)
m = GPy.models.GPRegression(X_train,Y_train[:, np.newaxis],kernel)
m.optimize()
prediction = m.predict(X_test,kernel)[0]
prediction_gp = m.predict(X_train,kernel)[0]

print 'accuracy', compute_accuracy(prediction, Y_test)*100/len(prediction),'%'

accuracy 39 %


In [23]:
#Doing random forest
clf = RandomForestClassifier(n_estimators=700)
clf.fit(X_train, Y_train)
prediction = clf.predict(X_test)
prediction_rf = clf.predict(X_train)
print 'accuracy', compute_accuracy(prediction, Y_test)*100/len(prediction),'%'

#other
sig_clf = CalibratedClassifierCV(clf, method="sigmoid", cv="prefit")
sig_clf.fit(X_train, Y_train)
prediction = sig_clf.predict(X_test)
prediction_sig = sig_clf.predict(X_train)
print 'accuracy', compute_accuracy(prediction, Y_test)*100/len(prediction),'%'

accuracy 25 %
accuracy 26 %


In [24]:
#SVM

clf = svm.SVC()
clf.fit(X_train, Y_train)
prediction = clf.predict(X_test)
prediction_SVM = clf.predict(X_train)
print 'accuracy', compute_accuracy(prediction, Y_test)*100/len(prediction),'%'

accuracy 15 %


In [25]:
#neural network
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)#solversgd, adam, lbgfs
clf.fit(X_train, Y_train)
prediction = clf.predict(X_test)
prediction_nn = clf.predict(X_train)
print 'accuracy', compute_accuracy(prediction, Y_test)*100/len(prediction),'%'


accuracy 15 %


In [29]:
#Using the outputs as inputs
final_train = np.zeros((len(X_train),len(X_train[0]+6)))
final_test = np.zeros((len(X_test),len(X_test[0]+6)))
final_train[:,:] = X_train